# 유튜브 크롤링 후 DB 적재 자동화

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import psycopg2
import re

In [4]:
#스크롤 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

# 해시태그 함수
def get_hashtag(outline_test) :
    # hash_tag가 안달려있는 영상이 있기때문에 오류 방지
    try:
        hash_tag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
        #print(hash_tag)
        hashtag_list.append(hash_tag)
            
    except Exception as e:
        hash_tag = ""
        #print(hash_tag)
        hashtag_list.append(hash_tag)

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    return command

뷔스티에 원피스, 롱패딩, 숏패딩, 롱코트, 숏코트, 잠옷, 홈웨어, 트위드자켓, 블레이저 자켓, 싱글 자켓, 크롭 자켓, 레더 자켓, 루즈핏, 테일러드 자켓, 린넨자켓, 반팔자켓, 라이더자켓, 오버핏자켓, 청자켓, 니트가디건, 볼레로, 아가일, 크롭가디건, 반팔가디건', '앙고라', '바람막이', '아노락', '윈드브레이커', '바시티자켓', '바시티점퍼', '야구점퍼', 후드집업, 투웨이집업, 여름아우터, 간절기아우터, 겨울아우터, 기모맨투맨, 니트맨투맨, 크롭 맨투맨', '남자 맨투맨', '반집업', 후드티, 남자후드, 꾸안꾸, 플리스, 반팔티, 크롭티, 블라우스, 와이셔츠, 셔츠, 체크셔츠, 타이다이, 옥스퍼드셔츠, 남방, 슬리브티, 니트, 남자니트, 조끼, '봄슬랙스', '여름슬랙스', '가을슬랙스', '겨울슬랙스', '간절기슬랙스', '데님팬츠', '연청', '진청', '중청', '흑청', '그레이진', '와이드팬츠', '밴딩바지', '스키니진', '린넨팬츠', '반바지', '하이웨스트', '로우웨스트', '부츠컷', '치노팬츠', '트레이닝복', '트레이닝바지', '조거팬츠', '카고바지', '일자팬츠', '냉장고바지', '멜빵바지', '오버롤팬츠', '찢청', '미니스커트', '미디스커트', '롱스커트', '플레어스커트', 'A라인 스커트', 'H라인 스커트', '청치마', '테니스스커트', '프릴스커트', '레더 스커트', '랩스커트', '데님스커트', '플리츠스커트', '미니원피스', '미디원피스', '롱원피스', '나시원피스', '코르셋', '셔츠원피스', '벨벳원피스', '꽃원피스', '골지원피스', '골지티', '쉬폰원피스', '쉬폰블라우스', '린넨원피스', '카라원피스', '랩원피스', '니트원피스', '후드원피스', '코듀로이', '반팔원피스', '점프슈트', '봄룩북', '봄옷쇼핑', '봄코디', '봄옷하울', '여름룩북', '여름옷쇼핑', '여름코디', '여름하울', '가을룩북', '가을옷쇼핑', '가을코디', '가을하울', '겨울룩북', '겨울옷쇼핑', '겨울코디', '겨울하울'

keyword_list = ['블라인드', '롤스크린', '커튼', '러그', '카페트', '매트', '쿠션', '방석', '소파커버', '조명', 'led평판등', '천장등', '장스탠드', '단스탠드', '데스크스탠드', '무드등', '장식조명', '벽조명', '센서등', '형광등', '집꾸미기', '방꾸미기', '인테리어', '선반', '행거', '옷장', '의자', '화장대', '콘솔', '전신거울', '침구세트', '이불', '베개', '토퍼', '패드', '매트리스커버']

'책추천', '책리뷰', '책소개', '인생책', '필독서', 

In [33]:
keyword_list = ['오디오북']

In [38]:
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

for keyword in keyword_list :

    # 키워드를 검색한 유튜브 주소 접속
    url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
    driver.get(url)

    #사이트 최대화
    driver.maximize_window()

    # 필터 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/tp-yt-paper-button').click()
    time.sleep(1)

    # 구분 -> 동영상 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[1]/a/div/yt-formatted-string').click()

    # 업로드 날짜 -> 올해 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[5]/a').click()


    # 스크롤 다운 50번 실행
    body = driver.find_element_by_tag_name('body')
    body.send_keys(Keys.PAGE_DOWN)

    for i in range(1,50):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    name = soup.select('a#video-title')
    video_url = soup.select('a#video-title')

    video_num_list = [] #영상번호
    name_list = [] #영상제목
    url_list = [] #영상주소

    for i in range(len(name)):
        video_num_list.append(i+1)
        name_list.append(name[i].text.strip())

    for i in video_url:
        url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))

    print(keyword, '영상 개수: ', len(name_list))

    # 업로드일 & 해시태그 가져오기
    day_list = [] #업로드일
    hashtag_list = [] #해시태그

    for i in range(0,len(name_list)):

        print(i+1, end=' ')
        
        if url_list[i].split('/')[3] == 'shorts':  #쇼츠

            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')
            
            driver.get(url_list[i])

            time.sleep(2)

            #더보기 클릭
            driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[1]/ytd-reel-video-renderer[1]/div[2]/ytd-reel-player-overlay-renderer/div[2]/div[1]/ytd-menu-renderer/yt-icon-button/button/yt-icon').click()
            time.sleep(2)

            #설명 클릭
            driver.find_element_by_xpath('/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer/tp-yt-paper-item').click()
            time.sleep(2)

            #업로드일 정보 가져오기
            day = driver.find_element_by_id("publish-time").text.split(':')[1].strip()
            day_list.append(day)

            time.sleep(2) 

            # 해시태그 가져오기
            outline_test = driver.find_element_by_class_name("style-scope ytd-reel-description-sheet-renderer").text
            get_hashtag(outline_test)

        else :

            driver.get(url_list[i])
                
            time.sleep(2)    

            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')

            time.sleep(3)  

            #업로드일 정보 가져오기
            #day = soup.select('#formatted-snippet-text > span:nth-child(3)')[0].text
            day = driver.find_element_by_id("info-strings").text.split('\n')[0]
            day_list.append(day)

            if day == "" :
                print('error in getting day')
                break

            time.sleep(2) 

            # 해시태그 가져오기
            outline_test = soup.find("yt-formatted-string", {"class":"content style-scope ytd-video-secondary-info-renderer"}).text    
            get_hashtag(outline_test)

    print('끝')

    day_list2 = [] #전처리 후 업로드일

    #업로드일 날짜형식으로 통일화
    for day in day_list : 
        day = day.replace('.', '')
        day = day.replace('최초 공개: ', '')
        day = day.replace('실시간 스트리밍 시작일: ', '')
        day = day.replace(' ', '-')
        day_list2.append(day)

    #df화
    youtubeDic = {
        '제목': name_list,
        '업로드일': day_list2,
        '주소': url_list,
        '해시태그' : hashtag_list
    }

    youtubeDf = pd.DataFrame(youtubeDic)

    # datetime 타입으로 변환
    youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])

    # 중복값제거
    youtubeDf = youtubeDf.drop_duplicates(['주소'], keep='first')

    print(youtubeDf.tail())


    #postgresql 접속
    conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()

    table = 'yt_books'

    #table에 데이터 넣기
    for i in range(len(youtubeDf)) :
        cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
        conn.commit()

    #db 연결 종료
    conn.cursor().close()
    conn.close()
    

오디오북 영상 개수:  125
1 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="info-strings"]"}
  (Session info: chrome=102.0.5005.63)
Stacktrace:
Backtrace:
	Ordinal0 [0x008ED953+2414931]
	Ordinal0 [0x0087F5E1+1963489]
	Ordinal0 [0x0076C6B8+837304]
	Ordinal0 [0x00799500+1021184]
	Ordinal0 [0x0079979B+1021851]
	Ordinal0 [0x007C6502+1205506]
	Ordinal0 [0x007B44E4+1131748]
	Ordinal0 [0x007C4812+1198098]
	Ordinal0 [0x007B42B6+1131190]
	Ordinal0 [0x0078E860+976992]
	Ordinal0 [0x0078F756+980822]
	GetHandleVerifier [0x00B5CC62+2510274]
	GetHandleVerifier [0x00B4F760+2455744]
	GetHandleVerifier [0x0097EABA+551962]
	GetHandleVerifier [0x0097D916+547446]
	Ordinal0 [0x00885F3B+1990459]
	Ordinal0 [0x0088A898+2009240]
	Ordinal0 [0x0088A985+2009477]
	Ordinal0 [0x00893AD1+2046673]
	BaseThreadInitThunk [0x75AAFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77267A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77267A4E+238]


In [8]:
for i in range(len(day_list)) :
    print(i, day_list[i], end=' / ')

day_list[18] = '2022-06-10'

0 2022. 5. 9. / 1 2021. 7. 22. / 2 2022. 2. 8. / 3 2021. 9. 10. / 4 최초 공개: 2022. 4. 7. / 5 2021. 11. 28. / 6 2021. 8. 25. / 7 2021. 7. 13. / 8 2022. 3. 12. / 9 2021. 10. 3. / 10 2022. 5. 27. / 11 2021. 6. 28. / 12 2021. 11. 23. / 13 2021. 7. 4. / 14 2021. 10. 15. / 15 2021. 9. 18. / 16 2022. 2. 13. / 17 2022. 1. 7. / 18 스트리밍 시작: 40분 전 / 19 2022. 5. 29. / 20 2022. 2. 4. / 21 최초 공개: 2022. 4. 1. / 22 2021. 12. 29. / 23 2022. 5. 23. / 24 최초 공개: 2021. 7. 4. / 25 2021. 10. 24. / 26 2021. 11. 4. / 27 2022. 4. 7. / 28 2022. 1. 2. / 29 2022. 2. 20. / 30 2022. 2. 6. / 31 2021. 6. 11. / 32 최초 공개: 2021. 7. 2. / 33 2021. 8. 19. / 34 2022. 2. 23. / 35 2021. 12. 16. / 36 2021. 6. 20. / 37 2021. 9. 5. / 38 2021. 11. 11. / 39 2022. 3. 29. / 40 2021. 8. 5. / 41 2021. 8. 30. / 42 2022. 3. 30. / 43 최초 공개: 2022. 4. 27. / 44 2021. 10. 17. / 45 2021. 12. 2. / 46 2021. 9. 14. / 47 2022. 5. 17. / 48 2021. 7. 22. / 49 2022. 5. 21. / 50 2021. 8. 27. / 51 2021. 7. 25. / 52 2021. 10. 3. / 53 2022. 3. 6. / 54 2022.

In [32]:
day_list2 = [] #전처리 후 업로드일

    #업로드일 날짜형식으로 통일화
for day in day_list : 
    day = day.replace('.', '')
    day = day.replace('최초 공개: ', '')
    day = day.replace('최초 공개일: ', '')
    day = day.replace('실시간 스트리밍 시작일: ', '')
    day = day.replace(' ', '-')
    day_list2.append(day)

    #df화
youtubeDic = {
    '제목': name_list,
    '업로드일': day_list2,
    '주소': url_list,
    '해시태그' : hashtag_list
}

youtubeDf = pd.DataFrame(youtubeDic)

    # datetime 타입으로 변환
youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])

    # 중복값제거
youtubeDf = youtubeDf.drop_duplicates(['주소'], keep='first')

print(youtubeDf.tail())


    #postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'yt_books'

    #table에 데이터 넣기
for i in range(len(youtubeDf)) :
    cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
    conn.commit()

    #db 연결 종료
conn.cursor().close()
conn.close()

                                                    제목       업로드일  \
144  지구를 지카자 프로젝트 11.26 아무것도 안사는날  필독서- 요셉의 작고 낡은 오... 2021-11-25   
145                 [이트너스 북튜브] 2021년도 4분기 필독서 - 『일의 격』 2021-12-16   
146                   4회)잊어 버렸던 만남4♡신앙인의 필독서,구속사 시리즈2권 2022-01-05   
147   [이트너스 북튜브] 2021년도 4분기 필독서 - 『최고의 변화는 어떻게 만들어지는가』 2021-12-01   
148                #마지막수업 #알퐁스도데 #초등필독서 #중등필독서 #세계명작단편 2022-01-12   

                                              주소  \
144  https://www.youtube.com/watch?v=bOXD1J3KSZk   
145  https://www.youtube.com/watch?v=2zcU3OGElN0   
146  https://www.youtube.com/watch?v=2nDnPzqw8HY   
147  https://www.youtube.com/watch?v=ok9ngTbH3Us   
148  https://www.youtube.com/watch?v=_NQ_Gygsuhc   

                                    해시태그  
144                                       
145                                       
146                                       
147                                       
148  #마지막수업 #알퐁스도데 #초등필독서 #중등필독서 #세계명작단편  


In [31]:
#varchar(500) type에 안들어가면 실행
b = 0

for i in range(len(hashtag_list)) :
    a = len(hashtag_list[i])

    if a > b :
        b = a
        print(i, b)

print(hashtag_list[141][:205])

0 12
2 14
3 26
4 33
13 40
19 70
24 85
63 91
68 112
75 200
96 356
#기업분할 #인적분할 #물적분할 #주린이 #필독서 #주식투자자필독서 #삼성전자 #삼성바이오로직스 #삼성 #현대차 #카카오 #기아 #셀트리온 #현대모비스 #삼성물산 #카카오뱅크 #신한지주 #카카오페이 #셀트리온헬스케어 #에코프로비엠 #펄어비스 #엘앤에프 #카카오게임즈 #위메이드 #셀트리온제약 #씨젠 #천보 #리노공업 #스튜디오드래곤 #알테오젠 #에스티팜 #에코프로


In [30]:
hashtag_list[132] = hashtag_list[132][:205]

# #############여기까지만#############

In [ ]:
#table 확인하기
cur.execute(select_command(table))
conn.commit()
cur.fetchall()

In [65]:
#db 연결 종료
conn.cursor().close()
conn.close()

create table yt_clothes (
	id serial,
	title varchar(100),
	date date,
	url varchar(200));